## 3장. 협업 필터링 추천 시스템

In [ ]:
##### (1)
# train set의 모든 가능한 사용자 pair의 Cosine similarities 계산
from sklearn.metrics.pairwise import cosine_similarity
# 코사인 유사도를 계산하는 sklearn.metrics.pairwise 라이브러리를 가져온다.

matrix_dummy = rating_matrix.copy().fillna(0)
# 코사인 유사도를 계산하기 위해 rating 값을 matrix_dummy에 복사한다. 그리고 코사인 유사도를 계산할 때 NaN값이 있으면 에러가 발생하므로
# NaN을 0으로 바꿔준다.
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
# 모든 사용자 간의 코사인 유사도를 구한다. 실행해보면 user_similarity는 (943X943)의 차원을 가지며 대각선이 1.0(자기 자신과의 유사도인)
# 대칭 매트릭스임을 볼 수 있다.
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)
# 나중에 필요한 값을 뽑아낼 수 있도록 user_similarity에 index를 지정해준다.


In [ ]:
# 주어진 영화의(movie_id) 가중평균 rating을 계산하는 함수
# 가중치는 주어진 사용자와 다른 사용자 간의 유사도(user_similarity)
def CF_simple(user_id, movie_id):
    # 주어진 user_id와 movie_id에 대한 예상 평점을 CF 알고리즘으로 계산해서 돌려주는 함수
    if movie_id in rating_matrix:
        # 주어진 movie_id가 rating_matrix에 있는지 확인한다. 앞에서 설명하였듯이 train/test set의 분할에 따라서 rating_matrix에 없는 영화가 있을 수 있는데, 이것이 에러를 발생시키는 것을 방지하기 위해서이다.
        sim_scores = user_similarity[user_id].copy()
        # 주어진 사용자(user_id)와 다른 사용자와의 유사도를 복사해서 가져온다.
        movie_ratings = rating_matrix[movie_id].copy()
        # 주어진 영화(movie_id)에 대한 모든 사용자의 평점을 복사해서 가져온다.
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        # 주어진 영화에 대해서 평가를 하지 않은 사용자의 위치를 확인한다. 이 사람들에 대해서는 가중평균 계산에서 빼기 위함이다.
        movie_ratings = movie_ratings.dropna()
        # 주어진 영화에 대한 평점 중에서 평가하지 않은 사람의 평점(NaN으로 표시된)을 제거한다.
        sim_scores = sim_scores.drop(none_rating_idx)
        # 주어진 영화를 평가하지 않은 사용자와의 유사도를 제거한다. 가중평균 계산할 때 필요가 없기 때문이다.
        mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
        # 주어진 영화에 대해서 평가한 각 사용자에 대해서 평점을 유사도로 가중평균한 예측치를 구한다.
        # 평가경향이 비슷한(유사도가 높은) 사용자에게 가중치를 더 주어서 평균을 계산하면 더 정확한 예측을 할 수 있음을 알 수 있다.
    else:
        mean_rating = 3.0
        # 주어진 영화가 rating_matrix에 없으면 기본값 3.0을 예측치로 한다.
    return mean_rating
        # 예측치를 돌려준다.

# 정확도 계산
score(CF_simple)
# 정확도를 계산한다.


### 연습문제 3-1
위의 코드를 수정해서 코사인 유사도 대신에 피어슨 상관계수(Pearson correlation coefficient)를 사용하는 코드를 작성하고 RMSE를 계산하세요.

## 3.4 이웃을 고려한 CF

- 이웃을 전체 사용자로 하는 대신에 유사도가 높은 사람만을 이웃으로 선정해서 이웃의 크기를 줄인다.
- 이웃을 정하는 기준 2가지
1. 이웃의 크기를 미리 정해놓고 추천 대상 사용자와 가장 유사한 K명을 선택하는 'K Nearest Neighbors(KNN)' 방법
2. 크기 대신 유사도의 기준(예를 들면 상관계수 0.8 이상)을 정해놓고 이 기준을 충족시키는 사용자를 이웃으로 정하는 Thresholding이다.

- Thresholding 방법이 KNN보다 정확하기는 하지만, 정해진 기준을 넘는 사용자가 없어서 추천을 하지 못하는 경우가 많기 때문에 KNN이 무난하게 많이 쓰인다.




In [ ]:
# 모델별 RMSE를 계산하는 함수
def score(model, neighbor_size=0):
    .
    .
    .
    y_pred = np.array([model](user, movie, neighbor_size) for (user, movie) in id_pairs)
    .
    .
    .

# Neighbor size를 정해서 예측치를 계산하는 함수
def cf_knn(user_id, movie_id, neighbor_size=0):
    if movie_id in rating_matrix:
        sim_scores = user_similarity[user_id].copy()
        movie_ratings = rating_matrix[movie_id].copy()